In [1]:
## import API keys from .env file in this folder
import os
from dotenv import load_dotenv
load_dotenv(".env")

openai_api_key    = os.getenv("openai_api_key")

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

In [ ]:
# !pip install pandas

In [3]:
import pandas as pd

df = pd.read_csv("documents/GSE246743_raw_counts.csv", index_col=0)
df = df.head(10)
df.head()

,KO_0.1,KO_0.2,KO_0.3,KO_4.1,KO_4.2,KO_4.3,KO_8.1,KO_8.2,KO_8.3,WT_0.1,WT_0.2,WT_0.3,WT_4.1,WT_4.2,WT_8.1,WT_8.2,WT_8.3
ENSMUSG00000102693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000064842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000051951,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
ENSMUSG00000102851,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000103377,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.T
df.head()

,ENSMUSG00000102693,ENSMUSG00000064842,ENSMUSG00000051951,ENSMUSG00000102851,ENSMUSG00000103377,ENSMUSG00000104017,ENSMUSG00000103025,ENSMUSG00000089699,ENSMUSG00000103201,ENSMUSG00000103147
KO_0.1,0,0,0,0,0,0,0,0,0,0
KO_0.2,0,0,0,0,0,0,0,0,0,0
KO_0.3,0,0,1,0,0,0,0,0,0,0
KO_4.1,0,0,0,0,0,0,0,0,0,0
KO_4.2,0,0,0,0,0,0,0,0,0,0


In [ ]:
# !pip install langchain-experimental

In [5]:
import pandas as pd
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['ENSMUSG00000051951'].mean()")

0.11764705882352941

In [6]:
tool.name

'python_repl_ast'

In [7]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'df' and want to know the standard deviation of 'ENSMUSG00000051951' column"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1IAeliANKPH4HKFZkqux2DJ0', 'function': {'arguments': '{"query":"df[\'ENSMUSG00000051951\'].std()"}', 'name': 'python_repl_ast'}, 'type': 'function'}]})

In [8]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, return_single=True)

(llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the standard deviation of 'ENSMUSG00000051951' column"
)

[{'query': "df['ENSMUSG00000051951'].std()"}]

In [9]:
resp = (llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the standard deviation of 'ENSMUSG00000051951' column"
)

In [10]:
resp[0]['query']

"df['ENSMUSG00000051951'].std()"

In [11]:
tool.invoke(resp[0]['query'])

0.3321055820775357

In [ ]:
# !pip install tabulate

In [12]:
df.head()

,ENSMUSG00000102693,ENSMUSG00000064842,ENSMUSG00000051951,ENSMUSG00000102851,ENSMUSG00000103377,ENSMUSG00000104017,ENSMUSG00000103025,ENSMUSG00000089699,ENSMUSG00000103201,ENSMUSG00000103147
KO_0.1,0,0,0,0,0,0,0,0,0,0
KO_0.2,0,0,0,0,0,0,0,0,0,0
KO_0.3,0,0,1,0,0,0,0,0,0,0
KO_4.1,0,0,0,0,0,0,0,0,0,0
KO_4.2,0,0,0,0,0,0,0,0,0,0


In [13]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

```
{df.head().to_markdown()}
```

Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])



In [14]:
prompt

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You have access to a pandas dataframe `df`. Here is the output of `df.head().to_markdown()`:\n\n```\n|        |   ENSMUSG00000102693 |   ENSMUSG00000064842 |   ENSMUSG00000051951 |   ENSMUSG00000102851 |   ENSMUSG00000103377 |   ENSMUSG00000104017 |   ENSMUSG00000103025 |   ENSMUSG00000089699 |   ENSMUSG00000103201 |   ENSMUSG00000103147 |\n|:-------|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|---------------------:|\n| KO_0.1 |                    0 |                    0 |                    0 |                    0 |                    0 |                    0 |                    0 |                    0 |                    0 |                    0 |\n| KO_0.2 |                    0 | 

In [15]:
code_chain = prompt | llm_with_tools | parser

code_chain.invoke({"question": "What's the standard deviation of ENSMUSG00000051951?"})

[{'query': "df['ENSMUSG00000051951'].std()"}]

In [16]:
def analyze_df(question):
    chain = (prompt | llm_with_tools | parser) 
    resp = chain.invoke({"question": question})
    return tool.invoke(resp[0]['query'])

In [17]:
analyze_df("What's the standard deviation of ENSMUSG00000051951?")

0.3321055820775357

In [18]:
analyze_df("What's the mean of ENSMUSG00000051951?")

0.11764705882352941